In [1]:
import random
from typing import Dict, Optional, List
import numpy as np

# Main functions

In [10]:


# Definizioni per gli operatori
UNARY_OPERATORS = ['sin', 'cos', 'exp', 'log', 'sqrt', 'tan', 'tanh', 'sinh', 'cosh', 'abs', 'log10', 'log2']
BINARY_OPERATORS = ['+', '-', '*', '/', '**', 'mod']

OPERATOR_WEIGHTS = {
    '+': 0.3,
    '-': 0.3,
    '*': 0.2,
    '/': 0.1,    # Divisioni meno frequenti
    '**': 0.05,  # Potenze rare
    'sin': 0.15,
    'cos': 0.15,
    'exp': 0.05, # Exponential molto raro
    'log': 0.1,
    'sqrt': 0.1,
    'tan': 0.05,
    'tanh': 0.05,
    'sinh': 0.05,
    'cosh': 0.05,
    'abs': 0.05,
    'log10': 0.05,
    'log2': 0.05,
    'mod': 0.05
}


# Configurazioni per la mutazione
class MutationConfig:
    MUTATION_WEIGHTS = {
        'SUBTREE': 0.7,
        'OPERATOR': 0.4,
        'VALUE': 0.2,
    }
    SUBTREE_DEPTH_RANGE = (1, 3)
    VALUE_STEP_FACTOR = 0.1
    MUTATION_DECAY = 0.5

# Funzione per limitare i valori
CLIP_MIN = -1e6
CLIP_MAX = 1e6

MAX_POWER = 5


def clip_value(value):
    return np.clip(value, CLIP_MIN, CLIP_MAX)

# Implementazione delle operazioni sicure
def safe_divide(a, b):
    return a / b if b != 0 else 0


def safe_sin(x):
    x = np.clip(x, -1000, 1000)  # Clip the input to avoid excessive values
    result = np.sin(x)
    return np.clip(result, -1000, 1000)

def safe_cos(x):
    x = np.clip(x, -1000, 1000)  # Clip the input to avoid excessive values
    result = np.cos(x)
    return np.clip(result, -1000, 1000)

def safe_sinh(x):
    x = np.clip(x, -100, 100)  # Limiting input to prevent overflow
    result = np.sinh(x)
    return np.clip(result, -1000, 1000)

def safe_cosh(x):
    x = np.clip(x, -100, 100)  # Limiting input to prevent overflow
    result = np.cosh(x)
    return np.clip(result, -1000, 1000)

def safe_tan(x):
    if np.isclose(np.mod(x, np.pi), np.pi / 2):  # tan is undefined at odd multiples of pi/2
        return float(10**6)
    x = np.clip(x, -1000, 1000)  # Clip the input to a manageable range
    result = np.tan(x)
    return np.clip(result, -1000, 1000)

def safe_log10(x):
    if x <= 0:
        return float(10**6)
    result = np.log10(x)
    return np.clip(result, -1000, 1000)

def safe_pow(base, exp):
    base = np.clip(base, -1000, 1000)  # Limiting base
    exp = np.clip(exp, -100, 100)    # Limiting exponent
    if base == 0 and exp < 0:
        return float(10**6)  # Return large penalty for invalid operation
    elif base < 0 and not np.all(np.isinteger(exp)):
        return float(10**6)
    try:
        result = np.power(base, exp)
        return np.clip(result, -1000, 1000)
    except ValueError:
        return float(10**6)

def safe_log2(x):
    if x <= 0:
        return float(10**6)
    result = np.log2(x)
    return np.clip(result, -1000, 1000)

def safe_mod(x, y):
    if y == 0:
        return float(10**6)
    x = np.clip(x, -1000, 1000)
    y = np.clip(y, -1000, 1000)
    result = np.mod(x, y)
    return np.clip(result, -1000, 1000)

def safe_tanh(x):
    x = np.clip(x, -1000, 1000)  # Limiting large inputs to avoid overflow
    result = np.tanh(x)
    return np.clip(result, -1000, 1000)

def safe_exp(x):
    x = np.clip(x, -100, 100)  # Limiting input to prevent overflow
    result = np.exp(x)
    return np.clip(result, -1000, 1000)

def safe_log(x):
    if x <= 0:
        return float(10**6)
    result = np.log(x)
    return np.clip(result, -1000, 1000)

def safe_sqrt(x):
    x = np.maximum(x, 0)  # Ensure input is non-negative
    result = np.sqrt(x)
    return np.clip(result, -1000, 1000)

def safe_abs(x):
    result = np.abs(x)
    return np.clip(result, -1000, 1000)

SAFE_OPERATIONS = {
    '+': lambda a, b: a + b,
    '-': lambda a, b: a - b,
    '*': lambda a, b: a * b,
    '/': safe_divide,
    'sin': safe_sin,
    'cos': safe_cos,
    'exp': safe_exp,
    'log': safe_log,
    'sqrt': safe_sqrt,
    'tan': safe_tan,
    'tanh': safe_tanh,
    'sinh': safe_sinh,
    'cosh': safe_cosh,
    'abs': safe_abs,
    'log10': safe_log10,
    'log2': safe_log2,
    '**': safe_pow,
    'mod': safe_mod
}

class Node:
    def __init__(self, value=None, op=None, left=None, right=None):
        self.value = value
        self.op = op
        self.left = left
        self.right = right

    def evaluate(self, x):
        try:
            if self.op in BINARY_OPERATORS:
                if not self.left or not self.right:
                    return 0
                left_val = self.left.evaluate(x)
                right_val = self.right.evaluate(x)
                return SAFE_OPERATIONS[self.op](left_val, right_val)
            elif self.op in UNARY_OPERATORS:
                if not self.left:
                    return 0
                operand_val = self.left.evaluate(x)
                return SAFE_OPERATIONS[self.op](operand_val)
            # Evaluate value of input variable 
            elif isinstance(self.value, str) and self.value.startswith("x["):
                index = int(self.value[2:-1])  # extract the index of input array, e.g. x[0] -> 0
                return clip_value(x[index])
            elif self.value is not None:
                return clip_value(self.value)
            
        except Exception:
            return 0

    def copy(self):  
        return Node(
            value=self.value,
            op=self.op,
            left=self.left.copy() if self.left else None,
            right=self.right.copy() if self.right else None,
        )

    def is_valid(self):
        if self.value is not None:
            return True
        if self.op in BINARY_OPERATORS:
            return self.left is not None and self.right is not None
        if self.op in UNARY_OPERATORS:
            return self.left is not None
        return False

    def __str__(self):
        if self.op in BINARY_OPERATORS:
            return f"({self.left} {self.op} {self.right})"
        elif self.op in UNARY_OPERATORS:
            return f"{self.op}({self.left})"
        else:
            return str(self.value)

def tree_to_string(tree: Node) -> str:
    return str(tree)

# def validate_tree(node: Node, n_variables: int):
#     if node is None or not node.is_valid():
#         # Sostituisci con un nodo foglia valido
#         return Node(generate_constant())
    
#     # Valida operatori unari
#     if node.op in UNARY_OPERATORS:
#         if node.op == 'log' and (node.left is None or node.left.evaluate([0]) <= 0):
#             return Node(generate_constant())
    
#     # Valida operatori binari
#     if node.op in BINARY_OPERATORS:
#         if node.op == '/' and (node.right is None or node.right.evaluate([0]) == 0):
#             return Node(generate_constant())
#         if node.op == '**' and (node.left is None or node.right is None or node.right.evaluate([0]) < 0):
#             return Node(generate_constant())

#     if node.left:
#         node.left = validate_tree(node.left, n_variables)
#     if node.right:
#         node.right = validate_tree(node.right, n_variables)
#     return node

def mutate(
    node: Node,
    mutation_prob: float,
    max_depth: int,
    n_variables: int,
    config: MutationConfig = MutationConfig(),):
    if random.random() > mutation_prob:
        return node

    mutation_type = random.choices(
        list(config.MUTATION_WEIGHTS.keys()),
        weights=list(config.MUTATION_WEIGHTS.values()),
    )[0]

    if mutation_type == 'SUBTREE':
        return create_random_tree(
            random.randint(*config.SUBTREE_DEPTH_RANGE),
            max_depth,
            n_variables
        )

    elif mutation_type == 'OPERATOR':
    # Cambia operatore con scelta pesata
        if node.op in UNARY_OPERATORS or node.op in BINARY_OPERATORS:
            node.op = choose_operator()


    elif mutation_type == 'VALUE':
        if node.value is not None and isinstance(node.value, (int, float)):
            step = (
                abs(node.value) * config.VALUE_STEP_FACTOR
                if node.value != 0
                else config.VALUE_STEP_FACTOR
            )
            node.value += random.gauss(0, step)

    if node.left:
        node.left = mutate(
            node.left,
            mutation_prob * config.MUTATION_DECAY,
            max_depth,
            n_variables,
            config=config,
        )
    if node.right:
        node.right = mutate(
            node.right,
            mutation_prob * config.MUTATION_DECAY,
            max_depth,
            n_variables,
            config=config,
        )

    return node


def choose_operator():
    operators = list(OPERATOR_WEIGHTS.keys())
    weights = list(OPERATOR_WEIGHTS.values())
    return random.choices(operators, weights=weights, k=1)[0]

def generate_constant():

    if random.random() < 0.5:
        return random.uniform(-1, 1)  # small value
    else:
        return random.uniform(-10, 10)  # larger value

def create_random_tree(depth: int, max_depth: int, n_variables: int) -> Node:
    if depth >= max_depth or (depth > 0 and random.random() < 0.5):
        # Nodo foglia
        if random.random() < 0.7:
            return Node(value=f"x[{random.randint(0, n_variables - 1)}]")
        else:
            return Node(generate_constant())

    # Scegli un operatore in base ai pesi
    op = choose_operator()
    if op in BINARY_OPERATORS:
        left = create_random_tree(depth + 1, max_depth, n_variables)
        right = create_random_tree(depth + 1, max_depth, n_variables)
        return Node(op=op, left=left, right=right)
    elif op in UNARY_OPERATORS:
        operand = create_random_tree(depth + 1, max_depth, n_variables)
        return Node(op=op, left=operand)
    else:
        # Se l'operatore non è definito in una delle due categorie, ad esempio:
        # Puoi decidere di trattarlo come unario o forzare la rigenerazione del nodo.
        operand = create_random_tree(depth + 1, max_depth, n_variables)
        return Node(op=op, left=operand)



# def crossover(parent1: Node, parent2: Node) -> Node:
#     """Effettua il crossover tra due individui."""
#     if random.random() < 0.9:
#         return parent1.copy()
#     return parent2.copy()

import random

def get_random_node(node: Node, include_root=True):
    """Raccoglie tutti i nodi dell'albero in una lista e restituisce un nodo casuale."""
    nodes = []
    def traverse(n):
        if n is None:
            return
        nodes.append(n)
        traverse(n.left)
        traverse(n.right)
    traverse(node)
    # Se non si vuole includere la radice, rimuovila dalla lista
    if not include_root and len(nodes) > 1:
        nodes = nodes[1:]
    return random.choice(nodes)

def crossover(parent1: Node, parent2: Node) -> Node:
    """
    Esegue il crossover tra due alberi:
    - Seleziona un nodo casuale (sottoalbero) in parent1
    - Seleziona un nodo casuale in parent2
    - Scambia i sottoalberi, restituendo una copia modificata di parent1
    """
    # Copia profonda del primo genitore per non modificare l'originale
    offspring = parent1.copy()

    # Seleziona un nodo casuale in offspring (inclusa la radice se desiderato)
    node1 = get_random_node(offspring, include_root=True)
    # Seleziona un nodo casuale in parent2 (si potrebbe anche scegliere se includere la radice o meno)
    node2 = get_random_node(parent2, include_root=True)

    # Scambia i sottoalberi: qui semplifichiamo assegnando il nodo2 copiato al posto del nodo1
    # Attenzione: questo metodo è semplificato, in una implementazione reale potresti voler gestire
    # il posizionamento del nuovo sottoalbero in maniera più accurata.
    if node1 is not None and node2 is not None:
        # Copia il nodo2 per evitare effetti collaterali
        new_subtree = node2.copy()

        # Ora, dobbiamo sostituire node1 con new_subtree nell'albero offspring.
        # Se node1 è la radice, l'intero albero offspring diventa new_subtree.
        if offspring == node1:
            offspring = new_subtree
        else:
            # Trova il padre di node1
            def replace_node(current, target, new_child):
                if current is None:
                    return False
                if current.left == target:
                    current.left = new_child
                    return True
                if current.right == target:
                    current.right = new_child
                    return True
                # Ricerca ricorsiva
                return replace_node(current.left, target, new_child) or replace_node(current.right, target, new_child)
            replace_node(offspring, node1, new_subtree)

    return offspring


def calculate_fitness(tree: Node, x: np.ndarray, y_true: np.ndarray) -> float:
    """Calcola la fitness di un individuo basata sull'errore quadratico medio."""
    try:
        y_pred = np.array([tree.evaluate(x[:, i]) for i in range(x.shape[1])])
        y_pred = np.clip(y_pred, CLIP_MIN, CLIP_MAX)
        y_pred = np.nan_to_num(y_pred, nan=0.0, posinf=CLIP_MAX, neginf=CLIP_MIN)  # Gestione di nan e inf
        return np.mean(np.square(y_true-y_pred))
    except Exception:
        return float('inf')

def calculate_complexity(node: Node) -> int:
    if node is None:
        return 0
    complexity = 1  # Base: 1 per ogni nodo
    
    # Definizione delle penalità
    HIGH_PENALTY_OPS = ['**', 'exp', 'log', 'mod']
    MEDIUM_PENALTY_OPS = ['/', 'tan', 'tanh', 'sqrt', 'log10', 'log2']
    LOW_PENALTY_OPS = ['+', '-', '*', 'sin', 'cos', 'sinh', 'cosh', 'abs']
    
    if node.op in HIGH_PENALTY_OPS:
        complexity += 3
    elif node.op in MEDIUM_PENALTY_OPS:
        complexity += 2
    elif node.op in LOW_PENALTY_OPS:
        complexity += 1
    complexity += calculate_complexity(node.left)
    complexity += calculate_complexity(node.right)
    return complexity




# def calculate_fitness(tree: Node, x: np.ndarray, y_true: np.ndarray, lambda_penalty: float = 0.0) -> float:
#     try:
#         # Calcola la predizione
#         y_pred = np.array([tree.evaluate(x[:, i]) for i in range(x.shape[1])])
#         y_pred = np.nan_to_num(y_pred, nan=float('inf'), posinf=CLIP_MAX, neginf=CLIP_MIN)
        
#         # Calcola l'MSE
#         mse = np.mean((y_pred - y_true) ** 2)
        
#         # Calcola la complessità
#         complexity = calculate_complexity(tree)
        
#         # Fitness totale
#         fitness = mse + lambda_penalty * complexity
#         return fitness
#     except Exception:
#         return float('inf')  # Penalizza alberi non validi


def tournament_selection(population: List[Node], fitness_scores: List[float], tournament_size: int = 3) -> Node:
    """Selezione tramite torneo."""
    competitors = random.sample(list(zip(population, fitness_scores)), tournament_size)
    winner = min(competitors, key=lambda x: x[1])  # Migliore fitness
    return winner[0]


def calculate_population_diversity(population: List[Node]) -> float:
    """Calcola la diversità della popolazione."""
    unique_trees = set(str(tree) for tree in population)
    return len(unique_trees) / len(population)

from collections import defaultdict
from typing import Optional

def maintain_diversity(population: List[Node], min_diversity: float, max_depth: int, n_variables: int):
    """Mantiene la diversità sostituendo individui simili."""
    current_diversity = calculate_population_diversity(population)
    # print(f"Diversità attuale: {current_diversity:.2f}")
    if current_diversity < min_diversity:
        # Raggruppa individui simili
        structures = defaultdict(list)
        for i, tree in enumerate(population):
            structures[str(tree)].append(i)

        # Sostituisci individui simili in eccesso
        for indices in structures.values():
            if len(indices) > 1 : # Tieni al massimo 2 di ogni struttura
                for idx in indices[1:]:
                    population[idx] = create_random_tree(
                        random.randint(1, max_depth),
                        max_depth,
                        n_variables
                    )


    return population
      



# Main loop

In [ ]:
# Parametri
POPULATION_SIZE = 500
MAX_DEPTH = 6
GENERATIONS = 50
MIN_DIVERSITY = 0.6
TOURNAMENT_SIZE = 2

# load dei dati
data = np.load('data/problem_1.npz')
x_train = data['x']
y_train = data['y']

N_VARIABLES = x_train.shape[0]


# # Inizializzazione della popolazione
population = [create_random_tree(0, MAX_DEPTH, N_VARIABLES) for _ in range(POPULATION_SIZE)]

## log print
print("Popolazione iniziale:")
for i, ind in enumerate(population):
    print(f"Individuo {i}: {ind}")

    
best_fitness = float('inf')

fitness_scores = [calculate_fitness(ind, x_train, y_train) for ind in population]

replacement_rate = 0.5
num_replacement = int(POPULATION_SIZE * replacement_rate)
for generation in range(GENERATIONS):
    parents = [tournament_selection(population, fitness_scores, TOURNAMENT_SIZE) for _ in range(2)]

    next_generation = []
    for i in range(0, POPULATION_SIZE, 2):
        child1 = crossover(parents[0], parents[1])
        child2 = crossover(parents[1], parents[0])

        child1 = mutate(child1, mutation_prob=0.4, max_depth=MAX_DEPTH, n_variables=N_VARIABLES)
        child2 = mutate(child2, mutation_prob=0.4, max_depth=MAX_DEPTH, n_variables=N_VARIABLES)

        next_generation.extend([child1, child2])
    offspring_fitness = [calculate_fitness(ind, x_train, y_train) for ind in next_generation]

    for child, fitness in zip(next_generation[:num_replacement], offspring_fitness[:num_replacement]):
        least_fit_idx = np.argmax(fitness_scores)
        if fitness < fitness_scores[least_fit_idx]:
            population[least_fit_idx] = child
            fitness_scores[least_fit_idx] = fitness

            if fitness < best_fitness:
                best_fitness = fitness
                best_tree = child

    # if calculate_population_diversity(population) < MIN_DIVERSITY:
    #     population = maintain_diversity(population, MIN_DIVERSITY, MAX_DEPTH, N_VARIABLES)


    diversity = calculate_population_diversity(population)
    print(f"Diversità: {div:.2f}")
    if generation % 5 == 0:
        if diversity < MIN_DIVERSITY:
            population = maintain_diversity(population, MIN_DIVERSITY, MAX_DEPTH, N_VARIABLES)


    print(f"Generazione {generation + 1}: Miglior fitness: {best_fitness}, Miglior individuo: {best_tree}")

print(f"Miglior individuo: {best_tree}")
print("Fitness sul training set:", calculate_fitness(best_tree, x_train, y_train))

        



# Working loop

In [11]:
import os
import glob


def genetic_algorithm(filepath: str):
    """
    Esegue l'algoritmo genetico su tutti i file 'problem_#.npz' presenti nella cartella
    indicata da `filepath` e salva le funzioni ottenute in un file Python denominato 's323914.py'.
    
    La funzione scrive nel file una definizione per ciascun problema, nel formato:
    
        import numpy as np
        
        def f1(x: np.ndarray) -> np.ndarray:  # mse: 0.1234
            return <formula>
    
    Dove il numero nella funzione (f1, f2, …) corrisponde al numero estratto dal nome del file
    (es. "problem_1.npz") e il commento indica il valore della fitness (mse) ottenuta.
    """
    # Costanti dell'algoritmo
    POPULATION_SIZE = 500
    MAX_DEPTH = 6
    GENERATIONS = 50
    MIN_DIVERSITY = 0.6
    TOURNAMENT_SIZE = 3
    replacement_rate = 0.5
    num_replacement = int(POPULATION_SIZE * replacement_rate)
    
    # Trova tutti i file con pattern "problem_*.npz" nella cartella specificata
    problem_files = glob.glob(os.path.join(filepath, "problem_*.npz"))
    solutions = []  # Lista dei tuple: (numero_problema, best_fitness, best_code)

    # Ciclo sui file dei problemi (ordinati per numero)
    for prob_file in sorted(problem_files):
        
        base = os.path.basename(prob_file)
        try:
            num_str = base.split('_')[1].split('.')[0]
            prob_number = int(num_str)
        except (IndexError, ValueError):
            print(f"Nome file non conforme, salto: {prob_file}")
            continue

        print(f"Risoluzione del problema {prob_number}...")
        # Escludi il problem_0
        if prob_number == 0:
            print(f"Salto il problema {prob_number} (problem_0 escluso).")
            continue

        # Carica i dati dal file .npz
        data = np.load(prob_file)
        x_train = data['x']
        y_train = data['y']
        N_VARIABLES = x_train.shape[0]

        # Inizializza la popolazione
        population = [create_random_tree(0, MAX_DEPTH, N_VARIABLES) for _ in range(POPULATION_SIZE)]

        best_fitness = float('inf')
        best_tree = None

        fitness_scores = [calculate_fitness(ind, x_train, y_train) for ind in population]

        # Ciclo evolutivo
        for generation in range(GENERATIONS):
            # Selezione dei due genitori tramite torneo
            parents = [tournament_selection(population, fitness_scores, TOURNAMENT_SIZE) for _ in range(2)]
            next_generation = []
            # Genera la prossima generazione con crossover e mutazione
            for i in range(0, POPULATION_SIZE, 2):
                child1 = crossover(parents[0], parents[1])
                child2 = crossover(parents[1], parents[0])
                child1 = mutate(child1, mutation_prob=0.4, max_depth=MAX_DEPTH, n_variables=N_VARIABLES)
                child2 = mutate(child2, mutation_prob=0.4, max_depth=MAX_DEPTH, n_variables=N_VARIABLES)
                next_generation.extend([child1, child2])
            offspring_fitness = [calculate_fitness(ind, x_train, y_train) for ind in next_generation]

            # Sostituisce alcuni individui della popolazione con i figli se migliorano la fitness
            for child, fitness in zip(next_generation[:num_replacement], offspring_fitness[:num_replacement]):
                least_fit_idx = np.argmax(fitness_scores)
                if fitness < fitness_scores[least_fit_idx]:
                    population[least_fit_idx] = child
                    fitness_scores[least_fit_idx] = fitness
                    if fitness < best_fitness:
                        best_fitness = fitness
                        best_tree = child

            diversity = calculate_population_diversity(population)
            print(f"Diversità: {diversity:.2f}")
            # Se ogni 5 generazioni la diversità scende sotto la soglia, si applica una strategia per mantenerla
            if generation % 2 == 0:
                if diversity < MIN_DIVERSITY:
                    population = maintain_diversity(population, MIN_DIVERSITY, MAX_DEPTH, N_VARIABLES)
            print(f"Generazione {generation + 1}: Miglior fitness: {best_fitness}, Miglior individuo: {best_tree}")

        print(f"Problema {prob_number} - Miglior individuo: {best_tree} con fitness: {best_fitness}")
        # Converte l'albero dell'individuo migliore in una stringa contenente una formula valida in Python.
        best_code = tree_to_string(best_tree)
        solutions.append((prob_number, best_fitness, best_code))

    # Scrive il file di output "s323914.py" con le funzioni ottenute
    output_filename = "s323914.py"
    with open(output_filename, "w") as f:
        f.write("import numpy as np\n\n")
        # Per ogni problema scrive la definizione della funzione (f1, f2, …)
        for prob_number, best_fitness, best_code in sorted(solutions, key=lambda x: x[0]):
            f.write(f"def f{prob_number}(x: np.ndarray) -> np.ndarray:  # mse: {best_fitness:.4f}\n")
            f.write("    return " + best_code + "\n\n")
    print(f"Soluzioni salvate in {output_filename}")


In [ ]:

genetic_algorithm("data")


Risoluzione del problema 1...
Diversità: 0.52
Generazione 1: Miglior fitness: 7.125940794232773e-34, Miglior individuo: sin(x[0])
Diversità: 0.58
Generazione 2: Miglior fitness: 7.125940794232773e-34, Miglior individuo: sin(x[0])
Diversità: 0.45
Generazione 3: Miglior fitness: 7.125940794232773e-34, Miglior individuo: sin(x[0])
Diversità: 0.56
Generazione 4: Miglior fitness: 7.125940794232773e-34, Miglior individuo: sin(x[0])
Diversità: 0.41
Generazione 5: Miglior fitness: 7.125940794232773e-34, Miglior individuo: sin(x[0])
Diversità: 0.57
Generazione 6: Miglior fitness: 7.125940794232773e-34, Miglior individuo: sin(x[0])
Diversità: 0.38
Generazione 7: Miglior fitness: 7.125940794232773e-34, Miglior individuo: sin(x[0])
Diversità: 0.64
Generazione 8: Miglior fitness: 7.125940794232773e-34, Miglior individuo: sin(x[0])
Diversità: 0.64
Generazione 9: Miglior fitness: 7.125940794232773e-34, Miglior individuo: sin(x[0])
Diversità: 0.64
Generazione 10: Miglior fitness: 7.125940794232773e-34

C:\Users\silvi\AppData\Local\Temp\ipykernel_26756\3797791689.py:94: RuntimeWarning: overflow encountered in power
  result = np.power(base, exp)


Diversità: 0.55
Generazione 5: Miglior fitness: 26912508981326.21, Miglior individuo: (-9.303247019064171 - log(x[0]))
Diversità: 0.76
Generazione 6: Miglior fitness: 26912508981326.21, Miglior individuo: (-9.303247019064171 - log(x[0]))
Diversità: 0.69
Generazione 7: Miglior fitness: 26912508981326.21, Miglior individuo: (-9.303247019064171 - log(x[0]))
Diversità: 0.73
Generazione 8: Miglior fitness: 26912508981326.21, Miglior individuo: (-9.303247019064171 - log(x[0]))
Diversità: 0.70
Generazione 9: Miglior fitness: 26912508981326.21, Miglior individuo: (-9.303247019064171 - log(x[0]))
Diversità: 0.66
Generazione 10: Miglior fitness: 26912508981326.21, Miglior individuo: (-9.303247019064171 - log(x[0]))
Diversità: 0.62
Generazione 11: Miglior fitness: 26912508981326.21, Miglior individuo: (-9.303247019064171 - log(x[0]))
Diversità: 0.62
Generazione 12: Miglior fitness: 24136573054682.09, Miglior individuo: (log(-5.387335978116187) * x[0])
Diversità: 0.59
Generazione 13: Miglior fitne

In [3]:
filepath = "data/problem_1.npz"
data = np.load(filepath)
x_train = data['x']
y_train = data['y']
print(x_train.shape)
print(y_train.shape)
print(x_train)
print(y_train)

filepath = "data/problem_0.npz"
data = np.load(filepath)
x_train = data['x']
y_train = data['y']
print(x_train.shape)
print(y_train.shape)
print(x_train)
print(y_train)



(2, 1000)
(1000,)
[[-1.24773177  2.79092518  2.73951968 ...  0.10033499  1.31381955
  -1.63225772]
 [ 0.24938969 -0.00342724 -0.68945101 ...  0.23088669 -0.46407131
  -0.2536372 ]]
[-1.19836925  2.79023974  2.61229694  0.92704309 -2.13334329 -2.6380194
  2.27335231  1.69257801  0.11802685  2.52820422  0.45925207 -1.73486157
 -0.19980504  0.72842124 -2.59416625  2.5085484   2.70161171 -2.2381064
 -1.69521149  0.32447683 -2.43617494 -2.31550077  1.80380563  0.55677356
 -1.22240006 -0.71303677  2.80964744  3.00706736  0.94725525 -1.13581223
 -2.87898911 -2.83774628 -2.62330738  1.16938062 -1.8845103   2.4452218
  2.42800907 -2.20762761  0.24142913 -1.19117969 -1.12168162 -2.26293825
  1.0842497  -0.19020732  0.1555053   0.45063789  1.41266154 -2.11832161
 -0.40610358  0.01592919 -1.29273788 -2.22354099 -2.00622662  0.3623328
  1.03116657 -0.73621145 -2.57148777  2.29926999 -2.14523807  3.15242889
 -1.74223917 -2.72577423  2.49552479  1.27451611 -3.0189541   0.3924791
  2.43694507 -3.28387

In [5]:
filepath8 = 'data/problem_8.npz'
# filepath4 = 'data/problem_4.npz'
# filepath5 = 'data/problem_5.npz'
# filepath6 = 'data/problem_6.npz'
# filepath7 = 'data/problem_7.npz'
# filepath8 = 'data/problem_8.npz'

genetic_algorithm(filepath8)
# genetic_algorithm(filepath4)
# genetic_algorithm(filepath5)
# genetic_algorithm(filepath6)
# genetic_algorithm(filepath7)
# genetic_algorithm(filepath8)


RecursionError: maximum recursion depth exceeded

In [106]:
import numpy as np
import random

# Operatori e funzioni supportate
OPERATORS = ['+', '-', '*', '/']
FUNCTIONS = ['sin', 'cos', 'exp', 'log']
class Node:
    def __init__(self, value, left=None, right=None):
        self.value = value
        self.left = left
        self.right = right

    def evaluate(self, x):
        try:
            if self.value in OPERATORS:
                left_val = self.left.evaluate(x)
                right_val = self.right.evaluate(x)
                # Gestione di divisioni per zero
                if self.value == '/' and np.any(right_val == 0):
                    return 1e10  # Penalità per divisione per zero
                result = eval(f"{left_val} {self.value} {right_val}")
                return np.nan_to_num(result, nan=1e10, posinf=1e10, neginf=1e10)
            elif self.value in FUNCTIONS:
                operand_val = self.left.evaluate(x)
                # Gestione di logaritmi negativi o nulli
                if self.value == 'log' and np.any(operand_val <= 0):
                    return 1e10  # Penalità per logaritmi invalidi
                result = eval(f"np.{self.value}({operand_val})")
                return np.nan_to_num(result, nan=1e10, posinf=1e10, neginf=1e10)
            elif isinstance(self.value, str) and self.value.startswith("x["):
                return eval(self.value)
            else:
                return float(self.value)
        except Exception as e:
            print(f"Errore durante la valutazione del nodo {self}: {e}")
            return 1e10  # Penalità per errori


    def __str__(self):
        if self.left and self.right:
            return f"({self.left} {self.value} {self.right})"
        elif self.left:
            return f"{self.value}({self.left})"
        else:
            return str(self.value)

def safe_generate_leaf(variable_count):
    """
    Genera una foglia valida (variabile o costante) garantendo che divisori e input a log siano validi.
    """
    if random.random() < 0.7:
        var_index = random.randint(0, variable_count - 1)
        return Node(f"x[{var_index}]")
    else:
        value = random.uniform(0.1, 1) if random.random() < 0.5 else random.uniform(-1, -0.1)  # Evita 0 per divisori
        return Node(f"{value:.3f}")

# Funzioni per generare alberi casuali
def generate_random_tree(variable_count, max_depth=3, ensure_all_vars=False):
    used_vars = set()

    def helper(depth):
        nonlocal used_vars
        if depth == 0 or (depth > 1 and random.random() < 0.3):
            leaf = safe_generate_leaf(variable_count)
            if leaf.value.startswith("x["):
                var_index = int(leaf.value[2:-1])
                used_vars.add(var_index)
            return leaf

        if random.random() < 0.5:
            left = helper(depth - 1)
            right = helper(depth - 1)
            return Node(random.choice(OPERATORS), left, right)
        else:
            operand = helper(depth - 1)
            return Node(random.choice(FUNCTIONS), operand)

    tree = helper(max_depth)


    # Se richiesto, garantire che tutte le variabili siano incluse
    if ensure_all_vars:
        missing_vars = [Node(f"x[{i}]") for i in range(variable_count) if i not in used_vars]
        for var in missing_vars:
            tree = Node('+', tree, var)

    return tree

# Funzione per verificare la conformità dei vincoli
# def ensure_all_variables(tree, variable_count):
#     used_vars = set()

#     def traverse(node):
#         if node is None:
#             return
#         if isinstance(node.value, str) and node.value.startswith("x["):
#             var_index = int(node.value[2:-1])
#             used_vars.add(var_index)
#         traverse(node.left)
#         traverse(node.right)

#     traverse(tree)

#     missing_vars = [Node(f"x[{i}]") for i in range(variable_count) if i not in used_vars]
#     for var in missing_vars:
#         tree = Node('+', tree, var)

#     return tree

# Inizializzazione della popolazione
def initialize_population(population_size, variable_count, max_depth=3):
    return [generate_random_tree(variable_count, max_depth, ensure_all_vars=False) for _ in range(population_size)]

# Calcolo della fitness
def calculate_fitness(tree, x, y_true):
    try:
        y_pred = np.array([tree.evaluate(x[:, i]) for i in range(x.shape[1])])
        y_pred = np.nan_to_num(y_pred)  # Converte NaN e inf in 0
        return np.mean((y_pred - y_true) ** 2)
    except Exception as e:
        print(f"Errore durante la valutazione della fitness per {tree}: {e}")
        return float('inf')

# Operazioni genetiche
def crossover(parent1, parent2):
    if random.random() < 0.5:
        return parent1
    return parent2

def mutate(tree, variable_count, max_depth=3):
    if random.random() < 0.5:
        return generate_random_tree(variable_count, max_depth, ensure_all_vars=False)
    return tree

In [107]:


# Inizializzazione
data = np.load('data/problem_0.npz')
x_train = data['x']
y_train = data['y']


# Parametri
POPULATION_SIZE = 100
VARIABLE_COUNT = x_train.shape[0]
MAX_DEPTH = 5

GENERATIONS = 50

# Inizializzazione della popolazione
population = initialize_population(POPULATION_SIZE, VARIABLE_COUNT, MAX_DEPTH)


# Esempio di utilizzo
POPULATION_SIZE = 50
MAX_DEPTH = 3
N_VARIABLES = 2

population = [create_random_tree(0, MAX_DEPTH, N_VARIABLES) for _ in range(POPULATION_SIZE)]
print("Diversità iniziale:", structural_diversity(population))

population = maintain_diversity(population, threshold=5.0)
print("Diversità mantenuta:", structural_diversity(population))


for generation in range(GENERATIONS):
    fitness_scores = [calculate_fitness(ind, x_train, y_train) for ind in population]

    # Selezione
    sorted_population = sorted(zip(population, fitness_scores), key=lambda x: x[1])
    parent1 = sorted_population[0][0]
    parent2 = sorted_population[1][0]

    # Crossover
    child = crossover(parent1, parent2)

    # Mutazione
    child = mutate(child, VARIABLE_COUNT, MAX_DEPTH)

    # Calcola la fitness del nuovo individuo
    child_fitness = calculate_fitness(child, x_train, y_train)

    # Sostituzione (steady state: sostituisce il peggiore)
    worst_index = fitness_scores.index(max(fitness_scores))
    population[worst_index] = child

    # Log della generazione
    best_fitness = min(fitness_scores)
    print(f"Generazione {generation + 1}: Miglior fitness: {best_fitness:.6f}")
    print(f"Individuo migliore: {population[fitness_scores.index(best_fitness)]}")

# Miglior individuo finale
fitness_scores = [calculate_fitness(ind, x_train, y_train) for ind in population]
best_index = fitness_scores.index(min(fitness_scores))
print(f"Miglior individuo: {population[best_index]} con fitness {fitness_scores[best_index]:.6f}")


TypeError: Node.__init__() got an unexpected keyword argument 'op'